### Altimetry Use Case example

##### Standard slicing from shed coordinates

In [5]:
from cfdmod.use_cases.altimetry import AltimetrySection, AltimetryShed, plot_surface, plot_profiles, plot_altimetry_profiles
import trimesh
import numpy as np
import pathlib

output_path = pathlib.Path("A:/Aerosim/Repositories/cfdmod/fixtures/altimetry/project_example/case_example")
surface_mesh: trimesh.Trimesh = trimesh.load_mesh("A:/Aerosim/Repositories/cfdmod/fixtures/altimetry/project_example/case_example/terrain.stl")

shed_start = np.array([-50,-50,820])
shed_end = np.array([50,50,820])

shed = AltimetryShed(shed_start, shed_end)

altimetry_section = AltimetrySection.from_points("example", shed_start, shed_end)
altimetry_section.slice_surface(surface_mesh)
altimetry_section.include_shed(shed)

plot_surface(surface_mesh, [altimetry_section], output_path)
plot_profiles([altimetry_section], output_path)
plot_altimetry_profiles(altimetry_section, output_path)

ValueError: string is not a file: A:/Aerosim/Repositories/cfdmod/fixtures/altimetry/project_example/case_example/terrain.stl

##### Generic Slicing from plane description

In [2]:
from cfdmod.use_cases.altimetry import plot_surface, plot_profiles, AltimetrySection
import trimesh
import numpy as np
import pathlib

output_path = pathlib.Path("A:/Aerosim/Repositories/cfdmod/fixtures/altimetry/project_example/case_example")
surface_mesh: trimesh.Trimesh = trimesh.load_mesh("A:/Aerosim/Repositories/cfdmod/fixtures/altimetry/project_example/case_example/terrain.stl")

plane_normal = np.array([1,0,0])
plane_origin = np.array([0,0,820])

altimetry_section = AltimetrySection("example", plane_origin, plane_normal)
altimetry_section.slice_surface(surface_mesh)

plot_surface(surface_mesh, [altimetry_section], output_path)
plot_profiles([altimetry_section], output_path)